In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_parquet("Shopping Carts.parquet")
df.reset_index(inplace=True)

In [4]:
df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index()
df = pd.merge(df,df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index())

### Welche Counties sind sich ähnlich in Hinblick auf die jeweiligen „Top 10“-Produkte?

In [5]:
df_3 = df.groupby(["county","product_name"]).agg(anzahl=("product_name","count")).reset_index()

In [6]:
counties = df_3.county.unique().tolist()
for i in counties:
    df_3.loc[df_3["county"] == i,"rank"] = df_3.loc[df_3["county"] == i].anzahl.rank(method="min",ascending=False)

In [7]:
df_4 = df_3[df_3["rank"].isin(range(11))]
df_4 = df_4.sort_values(["county","rank"])

In [8]:
df_4

,county,product_name,anzahl,rank
233,Alameda,Banana,352,1.0
218,Alameda,Bag of Organic Bananas,269,2.0
654,Alameda,Clementines,177,3.0
2351,Alameda,Organic Strawberries,168,4.0
1219,Alameda,Granny Smith Apples,139,5.0
...,...,...,...,...
576879,Yuba,Sparkling Water Grapefruit,668,6.0
573171,Yuba,Organic Half & Half,653,7.0
577190,Yuba,Strawberries,610,8.0
573178,Yuba,Organic Hass Avocado,598,9.0


In [8]:
df_13 = df_4.groupby("county").agg(coun=("county","count")).reset_index()

In [9]:
df_13[">10"] = df_13.coun > 10

In [10]:
df_4 = pd.merge(df_4,df_13)

In [11]:
df_4

,county,product_name,anzahl,rank,coun,>10
0,Alameda,Banana,352,1.0,11,True
1,Alameda,Bag of Organic Bananas,269,2.0,11,True
2,Alameda,Clementines,177,3.0,11,True
3,Alameda,Organic Strawberries,168,4.0,11,True
4,Alameda,Granny Smith Apples,139,5.0,11,True
...,...,...,...,...,...,...
574,Yuba,Sparkling Water Grapefruit,668,6.0,10,False
575,Yuba,Organic Half & Half,653,7.0,10,False
576,Yuba,Strawberries,610,8.0,10,False
577,Yuba,Organic Hass Avocado,598,9.0,10,False


In [25]:
df_4[df_4.county == "Alameda"]

,county,product_name,anzahl,rank,coun,>10
0,Alameda,Banana,352,1.0,11,True
1,Alameda,Bag of Organic Bananas,269,2.0,11,True
2,Alameda,Clementines,177,3.0,11,True
3,Alameda,Organic Strawberries,168,4.0,11,True
4,Alameda,Granny Smith Apples,139,5.0,11,True
5,Alameda,Sparkling Mineral Water,130,6.0,11,True
6,Alameda,Apples,122,7.0,11,True
7,Alameda,0% Greek Strained Yogurt,120,8.0,11,True
8,Alameda,Soda,106,9.0,11,True
9,Alameda,Organic Granny Smith Apple,102,10.0,11,True


In [11]:
fehler = df_4[df_4[">10"] == True].county.unique()

In [13]:
df_4[df_4.county == "Alameda"].tail(df_4[df_4.county == "Alameda"].coun.values[0]-10).index


Int64Index([10], dtype='int64')

In [12]:
for i in fehler:
    df_4.drop(df_4[df_4.county == i].tail(df_4[df_4.county == i].coun.values[0]-10).index,inplace=True)

In [13]:
df_4.head(50)

,county,product_name,anzahl,rank,coun,>10
0,Alameda,Banana,352,1.0,11,True
1,Alameda,Bag of Organic Bananas,269,2.0,11,True
2,Alameda,Clementines,177,3.0,11,True
3,Alameda,Organic Strawberries,168,4.0,11,True
4,Alameda,Granny Smith Apples,139,5.0,11,True
5,Alameda,Sparkling Mineral Water,130,6.0,11,True
6,Alameda,Apples,122,7.0,11,True
7,Alameda,0% Greek Strained Yogurt,120,8.0,11,True
8,Alameda,Soda,106,9.0,11,True
9,Alameda,Organic Granny Smith Apple,102,10.0,11,True


In [14]:
idk = {}
for i in counties:
    idk.update({f"{i}":1})

In [13]:
ableitung = []
test2 = []
for i in counties:
    
    for j in counties:
        score = 0
        
        for o in df_4.loc[df_4.county == i,("product_name","rank")].values:
            for z in df_4.loc[df_4.county == j,("product_name","rank")].values:
                if o[0] == z[0]:
                    score = score+1
                    if o[1] == z[1]:
                        score = score+1
        if score < 20:
        
         test2.append(i+j)
         if not(i+j in test2 and j+i in test2):
          ableitung.append([score,i,j])
         

In [68]:
test2

[{'Alameda', 'Alpine'},
 {'Alameda', 'Amador'},
 {'Alameda', 'Calaveras'},
 {'Alameda', 'Colusa'},
 {'Alameda', 'Contra Costa'},
 {'Alameda', 'Del Norte'},
 {'Alameda', 'El Dorado'},
 {'Alameda', 'Fresno'},
 {'Alameda', 'Glenn'},
 {'Alameda', 'Humboldt'},
 {'Alameda', 'Imperial'},
 {'Alameda', 'Inyo'},
 {'Alameda', 'Kern'},
 {'Alameda', 'Kings'},
 {'Alameda', 'Lake'},
 {'Alameda', 'Lassen'},
 {'Alameda', 'Los Angeles'},
 {'Alameda', 'Madera'},
 {'Alameda', 'Marin'},
 {'Alameda', 'Mariposa'},
 {'Alameda', 'Mendocino'},
 {'Alameda', 'Merced'},
 {'Alameda', 'Modoc'},
 {'Alameda', 'Mono'},
 {'Alameda', 'Monterey'},
 {'Alameda', 'Napa'},
 {'Alameda', 'Nevada'},
 {'Alameda', 'Orange'},
 {'Alameda', 'Placer'},
 {'Alameda', 'Plumas'},
 {'Alameda', 'Riverside'},
 {'Alameda', 'Sacramento'},
 {'Alameda', 'San Benito'},
 {'Alameda', 'San Bernardino'},
 {'Alameda', 'San Diego'},
 {'Alameda', 'San Francisco'},
 {'Alameda', 'San Joaquin'},
 {'Alameda', 'San Luis Obispo'},
 {'Alameda', 'San Mateo'},
 

In [36]:
idk["Alameda"]

{'Alpine': 5,
 'Amador': 4,
 'Calaveras': 5,
 'Colusa': 4,
 'Contra Costa': 3,
 'Del Norte': 3,
 'El Dorado': 5,
 'Fresno': 5,
 'Glenn': 5,
 'Humboldt': 2,
 'Imperial': 5,
 'Inyo': 6,
 'Kern': 6,
 'Kings': 3,
 'Lake': 6,
 'Lassen': 3,
 'Los Angeles': 3,
 'Madera': 6,
 'Marin': 3,
 'Mariposa': 5,
 'Mendocino': 7,
 'Merced': 5,
 'Modoc': 5,
 'Mono': 5,
 'Monterey': 6,
 'Napa': 8,
 'Nevada': 4,
 'Orange': 5,
 'Placer': 1,
 'Plumas': 3,
 'Riverside': 5,
 'Sacramento': 6,
 'San Benito': 5,
 'San Bernardino': 0,
 'San Diego': 3,
 'San Francisco': 6,
 'San Joaquin': 3,
 'San Luis Obispo': 6,
 'San Mateo': 3,
 'Santa Barbara': 3,
 'Santa Clara': 5,
 'Santa Cruz': 5,
 'Shasta': 5,
 'Sierra': 6,
 'Siskiyou': 5,
 'Solano': 5,
 'Sonoma': 6,
 'Stanislaus': 6,
 'Sutter': 5,
 'Tehama': 2,
 'Trinity': 1,
 'Tulare': 5,
 'Tuolumne': 4,
 'Ventura': 5,
 'Yolo': 4,
 'Yuba': 7}

In [15]:
ableitung.sort(reverse=True)

In [68]:
df_4[df_4.county == "Alameda"]

,county,product_name,anzahl,rank
233,Alameda,Banana,352,1.0
218,Alameda,Bag of Organic Bananas,269,2.0
654,Alameda,Clementines,177,3.0
2351,Alameda,Organic Strawberries,168,4.0
1219,Alameda,Granny Smith Apples,139,5.0
3083,Alameda,Sparkling Mineral Water,130,6.0
165,Alameda,Apples,122,7.0
1,Alameda,0% Greek Strained Yogurt,120,8.0
3047,Alameda,Soda,106,9.0
2106,Alameda,Organic Granny Smith Apple,102,10.0


In [148]:
df_4[df_4.county == "Orange"]

,county,product_name,anzahl,rank
330422,Orange,Banana,6030,1.0
330318,Orange,Bag of Organic Bananas,3654,2.0
342582,Orange,Organic Strawberries,3012,3.0
340759,Orange,Organic Baby Spinach,2792,4.0
341625,Orange,Organic Hass Avocado,2495,5.0
340738,Orange,Organic Avocado,2439,6.0
338131,Orange,Large Lemon,2001,7.0
338585,Orange,Limes,1981,8.0
342908,Orange,Organic Whole Milk,1967,9.0
342240,Orange,Organic Raspberries,1755,10.0


In [149]:
df_4[df_4.county == "Calaveras"]

,county,product_name,anzahl,rank
31399,Calaveras,Banana,7381,1.0
31281,Calaveras,Bag of Organic Bananas,5192,2.0
45259,Calaveras,Organic Strawberries,4177,3.0
43256,Calaveras,Organic Baby Spinach,3839,4.0
44197,Calaveras,Organic Hass Avocado,3542,5.0
43233,Calaveras,Organic Avocado,2937,6.0
40238,Calaveras,Large Lemon,2312,7.0
40737,Calaveras,Limes,2251,8.0
50558,Calaveras,Strawberries,2140,9.0
44890,Calaveras,Organic Raspberries,2137,10.0


In [67]:
df_4[df_4.county == "San Bernardino"]

,county,product_name,anzahl,rank
389620,San Bernardino,Original Rice Krispies Treats,6,1.0
389557,San Bernardino,April Fresh Liquid Fabric Softener,4,2.0
389589,San Bernardino,Fabric Softener Dryer Sheet Outdoor Fresh 160C...,4,2.0
389610,San Bernardino,Milk Chocolate Covered Raisins,4,2.0
389615,San Bernardino,Old Fashioned Rods Pretzels,4,2.0
389616,San Bernardino,Organic 2% Reduced Fat Milk,4,2.0
389632,San Bernardino,"Select-A-Size Paper Towels, White, 2 Huge Roll...",4,2.0
389645,San Bernardino,White Antibacterial Deodorant Soap Bars,4,2.0
389647,San Bernardino,With Raspberry Lemonade,4,2.0
389570,San Bernardino,Chocolate Chip Cookie Dough,3,10.0


maybe gruppen bilden oder über alle

In [17]:
ableitung

[[18, 'Imperial', 'Santa Clara'],
 [18, 'Calaveras', 'Orange'],
 [18, 'Calaveras', 'Glenn'],
 [17, 'Lake', 'Monterey'],
 [17, 'Kern', 'Monterey'],
 [17, 'Glenn', 'Sutter'],
 [17, 'Fresno', 'Orange'],
 [16, 'Santa Clara', 'Siskiyou'],
 [16, 'Orange', 'Sutter'],
 [16, 'Mariposa', 'Santa Cruz'],
 [16, 'Mariposa', 'Mono'],
 [16, 'Glenn', 'Orange'],
 [16, 'Calaveras', 'Sutter'],
 [16, 'Calaveras', 'Siskiyou'],
 [16, 'Calaveras', 'Plumas'],
 [16, 'Calaveras', 'Monterey'],
 [16, 'Calaveras', 'Kern'],
 [16, 'Calaveras', 'Fresno'],
 [15, 'Siskiyou', 'Ventura'],
 [15, 'Siskiyou', 'Sutter'],
 [15, 'Orange', 'Siskiyou'],
 [15, 'Orange', 'Santa Cruz'],
 [15, 'Monterey', 'Stanislaus'],
 [15, 'Monterey', 'Orange'],
 [15, 'Mono', 'Santa Cruz'],
 [15, 'Mariposa', 'Orange'],
 [15, 'Madera', 'San Francisco'],
 [15, 'Madera', 'Monterey'],
 [15, 'Madera', 'Mendocino'],
 [15, 'Kern', 'Stanislaus'],
 [15, 'Kern', 'Orange'],
 [15, 'Imperial', 'Ventura'],
 [15, 'Glenn', 'Siskiyou'],
 [15, 'Glenn', 'San Benito'

In [14]:
import plotly.graph_objects as go

import pandas as pd

In [45]:
for i in ableitung:
    print(i)

[18, 'Santa Clara', 'Imperial']
[18, 'Orange', 'Calaveras']
[18, 'Imperial', 'Santa Clara']
[18, 'Glenn', 'Calaveras']
[18, 'Calaveras', 'Orange']
[18, 'Calaveras', 'Glenn']
[17, 'Sutter', 'Glenn']
[17, 'Orange', 'Fresno']
[17, 'Monterey', 'Lake']
[17, 'Monterey', 'Kern']
[17, 'Lake', 'Monterey']
[17, 'Kern', 'Monterey']
[17, 'Glenn', 'Sutter']
[17, 'Fresno', 'Orange']
[16, 'Sutter', 'Orange']
[16, 'Sutter', 'Calaveras']
[16, 'Siskiyou', 'Santa Clara']
[16, 'Siskiyou', 'Calaveras']
[16, 'Santa Cruz', 'Mariposa']
[16, 'Santa Clara', 'Siskiyou']
[16, 'Plumas', 'Calaveras']
[16, 'Orange', 'Sutter']
[16, 'Orange', 'Glenn']
[16, 'Monterey', 'Calaveras']
[16, 'Mono', 'Mariposa']
[16, 'Mariposa', 'Santa Cruz']
[16, 'Mariposa', 'Mono']
[16, 'Kern', 'Calaveras']
[16, 'Glenn', 'Orange']
[16, 'Fresno', 'Calaveras']
[16, 'Calaveras', 'Sutter']
[16, 'Calaveras', 'Siskiyou']
[16, 'Calaveras', 'Plumas']
[16, 'Calaveras', 'Monterey']
[16, 'Calaveras', 'Kern']
[16, 'Calaveras', 'Fresno']
[15, 'Ventura'

In [20]:
df_4.head(20)

,county,product_name,anzahl,rank,coun,>10
0,Alameda,Banana,352,1.0,11,True
1,Alameda,Bag of Organic Bananas,269,2.0,11,True
2,Alameda,Clementines,177,3.0,11,True
3,Alameda,Organic Strawberries,168,4.0,11,True
4,Alameda,Granny Smith Apples,139,5.0,11,True
5,Alameda,Sparkling Mineral Water,130,6.0,11,True
6,Alameda,Apples,122,7.0,11,True
7,Alameda,0% Greek Strained Yogurt,120,8.0,11,True
8,Alameda,Soda,106,9.0,11,True
9,Alameda,Organic Granny Smith Apple,102,10.0,11,True


#### cosinus


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [21]:
ableitung1 = []
test2 = []
for i in idk:
    test = {}
    for j in idk:
        score = 0
        score = cosine_similarity(df_8[df_8.county == i].drop("county",axis=1).values,df_8[df_8.county == j].drop("county",axis=1).values).diagonal().sum()
     
        if score < 10:
        #  test.update({f"{j}":score})
         test2.append(i+j)
         if not(i+j in test2 and j+i in test2):
          ableitung1.append([score,i,j])

                
    #print(score, test)
    #idk.update({i:test})

In [22]:
ableitung1.sort(reverse=True)

In [23]:
ableitung1

[[9.499999999999998, 'Imperial', 'Santa Clara'],
 [9.499999999999998, 'Calaveras', 'Orange'],
 [8.999999999999998, 'Mariposa', 'Mono'],
 [8.999999999999998, 'Kern', 'Monterey'],
 [8.999999999999998, 'Glenn', 'Sutter'],
 [8.999999999999998, 'Calaveras', 'Glenn'],
 [8.499999999999998, 'Orange', 'Sutter'],
 [8.499999999999998, 'Mariposa', 'Santa Cruz'],
 [8.499999999999998, 'Lake', 'Monterey'],
 [8.499999999999998, 'Kern', 'Stanislaus'],
 [8.499999999999998, 'Glenn', 'Orange'],
 [8.499999999999998, 'Fresno', 'San Benito'],
 [8.499999999999998, 'Fresno', 'Orange'],
 [8.499999999999998, 'Calaveras', 'Sutter'],
 [8.499999999999998, 'Calaveras', 'Plumas'],
 [8.499999999999998, 'Calaveras', 'Monterey'],
 [8.499999999999998, 'Calaveras', 'Fresno'],
 [8.499999999999998, 'Alpine', 'San Benito'],
 [7.999999999999998, 'Siskiyou', 'Ventura'],
 [7.999999999999998, 'Siskiyou', 'Sutter'],
 [7.999999999999998, 'Santa Clara', 'Siskiyou'],
 [7.999999999999998, 'San Benito', 'Sutter'],
 [7.999999999999998,

In [66]:
df_4[df_4.county == "Mariposa"]

,county,product_name,anzahl,rank
239904,Mariposa,Banana,3466,1.0
239805,Mariposa,Bag of Organic Bananas,2405,2.0
251037,Mariposa,Organic Strawberries,1746,3.0
249417,Mariposa,Organic Baby Spinach,1592,4.0
250197,Mariposa,Organic Hass Avocado,1372,5.0
255363,Mariposa,Strawberries,1256,6.0
249399,Mariposa,Organic Avocado,1174,7.0
246987,Mariposa,Large Lemon,1129,8.0
251332,Mariposa,Organic Whole Milk,1016,9.0
247388,Mariposa,Limes,1010,10.0


In [65]:
df_4[df_4.county == "Mono"]

,county,product_name,anzahl,rank
292269,Mono,Banana,502,1.0
292249,Mono,Bag of Organic Bananas,484,2.0
294973,Mono,Organic Strawberries,335,3.0
294182,Mono,Organic Baby Spinach,274,4.0
294544,Mono,Organic Hass Avocado,243,5.0
295936,Mono,Strawberries,181,6.0
294822,Mono,Organic Raspberries,166,7.0
293661,Mono,Large Lemon,159,8.0
295112,Mono,Organic Whole Milk,158,9.0
294629,Mono,Organic Lemon,156,10.0


In [20]:
df_7 = df_4[["county","product_name","rank"]].copy()
df_8 = pd.get_dummies(df_7,columns=["product_name","rank"])

In [19]:
df_8

,county,product_name_0% Greek Strained Yogurt,product_name_100% Prune Juice,product_name_100% Raw Coconut Water,product_name_100% Whole Wheat Bread,product_name_2% Reduced Fat Milk,product_name_All American Salad,product_name_Almond Breeze Original Almond Milk,product_name_Apple Honeycrisp Organic,product_name_Apples,...,rank_1.0,rank_2.0,rank_3.0,rank_4.0,rank_5.0,rank_6.0,rank_7.0,rank_8.0,rank_9.0,rank_10.0
233,Alameda,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
218,Alameda,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
654,Alameda,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2351,Alameda,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1219,Alameda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576879,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
573171,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
577190,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
573178,Yuba,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
Alameda = df_8[df_8.county == "Alameda"].drop("county",axis=1)

In [63]:
San_Bernardino = df_8[df_8.county == "San Bernardino"].drop("county",axis=1)


In [21]:
Alameda

,product_name_0% Greek Strained Yogurt,product_name_100% Prune Juice,product_name_100% Raw Coconut Water,product_name_100% Whole Wheat Bread,product_name_2% Reduced Fat Milk,product_name_All American Salad,product_name_Almond Breeze Original Almond Milk,product_name_Apple Honeycrisp Organic,product_name_Apples,product_name_April Fresh Liquid Fabric Softener,...,rank_1.0,rank_2.0,rank_3.0,rank_4.0,rank_5.0,rank_6.0,rank_7.0,rank_8.0,rank_9.0,rank_10.0
233,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
218,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
654,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2351,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
165,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [29]:
#Alameda = sparse.csr_matrix(Alameda)

In [26]:
Monterey = df_8[df_8.county == "Monterey"].drop("county",axis=1)
#Calaveras = sparse.csr_matrix(Calaveras)

In [27]:
Marin = df_8[df_8.county == "Marin"].drop("county",axis=1)

In [23]:
Glenn.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [21]:
Calaveras.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [36]:
pd.concat([Calaveras,Monterey])

,product_name_0% Greek Strained Yogurt,product_name_100% Prune Juice,product_name_100% Raw Coconut Water,product_name_100% Whole Wheat Bread,product_name_2% Reduced Fat Milk,product_name_All American Salad,product_name_Almond Breeze Original Almond Milk,product_name_Apple Honeycrisp Organic,product_name_Apples,product_name_April Fresh Liquid Fabric Softener,...,rank_1.0,rank_2.0,rank_3.0,rank_4.0,rank_5.0,rank_6.0,rank_7.0,rank_8.0,rank_9.0,rank_10.0
31399,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
31281,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
45259,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
43256,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
44197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
43233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
40238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
40737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
50558,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [46]:
Alameda.drop(Alameda.tail(1).index,inplace=True)

In [69]:
Alameda = sparse.csr_matrix(Alameda.values)
Orange = sparse.csr_matrix(Orange.values)
#Alameda.values


AttributeError: values not found

In [41]:
Alameda.values*Calaveras.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [41]:
cosine_similarity(Alameda,pd.concat([Marin,Monterey]))

In [43]:
df_10.sum()

13.999999999999996

In [29]:
cosine_similarity(Monterey,Marin)

array([[0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0.5, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5]])

In [64]:
u, s, vh = np.linalg.svd(cosine_similarity(Alameda,San_Bernardino))

In [65]:
X2D = cosine_similarity(Alameda,San_Bernardino) @ vh[1,:].T

In [66]:
X2D.sum()

1.414213562373095

In [75]:
ableitung = []
test2 = []
for i in idk:
    test = {}
    for j in idk:
         for k in idk:   
                score = 0
            
                for o in df_4.loc[df_4.county == i,("product_name","rank")].values:
                    for z in df_4.loc[df_4.county == j,("product_name","rank")].values:
                        for l in df_4.loc[df_4.county == k,("product_name","rank")].values:
                            if o[0] == z[0] == l[0]:
                                score = score+1
                                if o[1] == z[1] == l[1]:
                                    score = score+1
                if score < 30:
                    #test.update({f"{j}":score})
                    test2.append(i+j)
                    if not(i+j+k in test2 and j+i+k in test2):
                     ableitung.append([score,i,j])

                
    # print(score, test)
    # idk.update({i:test})

KeyboardInterrupt: 

In [ ]:
ableitung = []
test2 = []
for i in idk:
    test = {}
    for j in idk:
        score = 0
        score = cosine_similarity(df_8[df_8.county == i].drop("county",axis=1).values,df_8[df_8.county == j].drop("county",axis=1).values).diagonal().sum()
     
        if score < 10:
        #  test.update({f"{j}":score})
         test2.append(i+j)
         if not(i+j in test2 and j+i in test2):
          ableitung.append([score,i,j])

                
    #print(score, test)
    #idk.update({i:test})

#### weiter

In [16]:
ableitung

[[18, 'Imperial', 'Santa Clara'],
 [18, 'Calaveras', 'Orange'],
 [18, 'Calaveras', 'Glenn'],
 [17, 'Lake', 'Monterey'],
 [17, 'Kern', 'Monterey'],
 [17, 'Glenn', 'Sutter'],
 [17, 'Fresno', 'Orange'],
 [16, 'Santa Clara', 'Siskiyou'],
 [16, 'Orange', 'Sutter'],
 [16, 'Mariposa', 'Santa Cruz'],
 [16, 'Mariposa', 'Mono'],
 [16, 'Glenn', 'Orange'],
 [16, 'Calaveras', 'Sutter'],
 [16, 'Calaveras', 'Siskiyou'],
 [16, 'Calaveras', 'Plumas'],
 [16, 'Calaveras', 'Monterey'],
 [16, 'Calaveras', 'Kern'],
 [16, 'Calaveras', 'Fresno'],
 [15, 'Siskiyou', 'Ventura'],
 [15, 'Siskiyou', 'Sutter'],
 [15, 'Orange', 'Siskiyou'],
 [15, 'Orange', 'Santa Cruz'],
 [15, 'Monterey', 'Stanislaus'],
 [15, 'Monterey', 'Orange'],
 [15, 'Mono', 'Santa Cruz'],
 [15, 'Mariposa', 'Orange'],
 [15, 'Madera', 'San Francisco'],
 [15, 'Madera', 'Monterey'],
 [15, 'Madera', 'Mendocino'],
 [15, 'Kern', 'Stanislaus'],
 [15, 'Kern', 'Orange'],
 [15, 'Imperial', 'Ventura'],
 [15, 'Glenn', 'Siskiyou'],
 [15, 'Glenn', 'San Benito'

In [18]:
df_map = pd.DataFrame(ableitung,columns=["score","county1","county2"])

In [17]:
df_table = df_4.iloc[:, 0:4]
df_table

,county,product_name,anzahl,rank
0,Alameda,Banana,352,1.0
1,Alameda,Bag of Organic Bananas,269,2.0
2,Alameda,Clementines,177,3.0
3,Alameda,Organic Strawberries,168,4.0
4,Alameda,Granny Smith Apples,139,5.0
...,...,...,...,...
574,Yuba,Sparkling Water Grapefruit,668,6.0
575,Yuba,Organic Half & Half,653,7.0
576,Yuba,Strawberries,610,8.0
577,Yuba,Organic Hass Avocado,598,9.0


In [19]:
from urllib.request import urlopen
import json
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    geojsn = json.load(response)

In [20]:
a = geojsn["features"].__len__()
for i in range(a,0,-1):
    if geojsn["features"][i-1]["properties"]["STATE"] != "06":
        geojsn["features"].pop(i-1)

In [30]:
df_map.county1.unique()

array(['Imperial', 'Calaveras', 'Lake', 'Kern', 'Glenn', 'Fresno',
       'Santa Clara', 'Orange', 'Mariposa', 'Siskiyou', 'Monterey',
       'Mono', 'Madera', 'Contra Costa', 'Alpine', 'San Benito',
       'Sacramento', 'Los Angeles', 'Inyo', 'Sutter', 'Santa Cruz',
       'Santa Barbara', 'San Luis Obispo', 'San Francisco', 'Riverside',
       'Plumas', 'Merced', 'Mendocino', 'Marin', 'Tuolumne', 'Stanislaus',
       'Solano', 'San Mateo', 'Nevada', 'Modoc', 'Colusa', 'Amador',
       'Tulare', 'Sonoma', 'Sierra', 'Shasta', 'Kings', 'Del Norte',
       'Ventura', 'Napa', 'San Joaquin', 'Alameda', 'San Diego', 'Lassen',
       'El Dorado', 'Yolo', 'Trinity', 'Tehama', 'Placer', 'Humboldt',
       'San Bernardino'], dtype=object)

In [31]:
df_map.county2.unique()

array(['Santa Clara', 'Orange', 'Glenn', 'Monterey', 'Sutter', 'Siskiyou',
       'Santa Cruz', 'Mono', 'Plumas', 'Kern', 'Fresno', 'Ventura',
       'Stanislaus', 'San Francisco', 'Mendocino', 'San Benito',
       'Mariposa', 'Santa Barbara', 'San Luis Obispo', 'Sacramento',
       'Madera', 'Lake', 'Los Angeles', 'Sierra', 'Sonoma', 'Solano',
       'Imperial', 'Tulare', 'Yolo', 'Riverside', 'Tuolumne', 'Modoc',
       'San Mateo', 'Nevada', 'Merced', 'Marin', 'Colusa', 'Calaveras',
       'Yuba', 'Shasta', 'Inyo', 'Contra Costa', 'San Joaquin', 'Kings',
       'Del Norte', 'Napa', 'San Diego', 'El Dorado', 'Amador', 'Trinity',
       'Tehama', 'Alpine', 'Humboldt', 'Placer', 'Lassen',
       'San Bernardino'], dtype=object)

In [61]:
np.append(df_map.loc[df_map.county2 == "Placer"].county1.values,df_map.loc[df_map.county1 == "Placer"].county2.values)

array(['Marin', 'Inyo', 'Orange', 'Nevada', 'Napa', 'Monterey', 'Mono',
       'Merced', 'Mendocino', 'Mariposa', 'Madera', 'Los Angeles', 'Lake',
       'Kings', 'Kern', 'Imperial', 'Humboldt', 'Glenn', 'Fresno',
       'Contra Costa', 'Colusa', 'Calaveras', 'Amador', 'Alpine', 'Modoc',
       'El Dorado', 'Del Norte', 'Alameda', 'Lassen', 'Sonoma', 'Yuba',
       'Yolo', 'Ventura', 'Tuolumne', 'Tulare', 'Sutter', 'Stanislaus',
       'Solano', 'Siskiyou', 'Sierra', 'Shasta', 'Santa Cruz',
       'Santa Clara', 'Santa Barbara', 'San Mateo', 'San Luis Obispo',
       'San Joaquin', 'San Francisco', 'San Benito', 'Sacramento',
       'Riverside', 'Plumas', 'Tehama', 'San Diego', 'Trinity',
       'San Bernardino'], dtype=object)

In [49]:
[df_map.loc[df_map.county1 == "Placer"].score.values.tolist() +df_map.loc[df_map.county2 == "Placer"].score.values.tolist() ][0].__len__()

56

In [37]:
df_map.loc[df_map.county1 == "Placer"].county2.values.append( df_map.loc[df_map.county2 == "Placer"].county1.values)

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [33]:
fig = go.Figure()
fig.add_trace(
    go.Choroplethmapbox(
        geojson= geojsn,
        featureidkey= "properties.NAME",
        locations= df_map.county1.values,
        z= df_map.score,
        showlegend=False,
        showscale=True,
        marker_opacity=0.5,
        marker_line_width=0
    )
)
fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=5, mapbox_center = {"lat": 37, "lon": -120}, height= 600)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [62]:

fig = go.Figure()
fig.add_trace(
    go.Choroplethmapbox(
        geojson= geojsn,
        featureidkey= "properties.NAME",
        locations= df_map.county1.values,
        z= df_map.score,
        showlegend=False,
        showscale=True,
        marker_opacity=0.5,
        marker_line_width=0
    )
)
fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=5, mapbox_center = {"lat": 37, "lon": -120}, height= 600)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    updatemenus= [
        go.layout.Updatemenu(
            active= 0,
            buttons= [
                dict(
                    label= county,
                    method= "update",
                    args= [
                        dict(
                            z = [np.append(df_map.loc[df_map.county2 == county].score.values,df_map.loc[df_map.county1 == county].score.values)],
                            locations= [np.append(df_map.loc[df_map.county2 == county].county1.values,df_map.loc[df_map.county1 == county].county2.values)]
                        )
                    ]

                )
                for county in df_map.county1.unique().tolist()
            ]            
        )
    ]
)
fig.show()

In [78]:
fig = make_subplots(cols= 2, rows=1, specs= [[dict(type= "table"), dict(type= "choroplethmapbox")]])

fig.add_trace(
  go.Table(
    columnwidth= [2, 4, 2, 2, 2],
    header=dict(
        values=list(df_table.columns),
        fill_color= px.colors.qualitative.G10[4],
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "white", size=15),
        height= 35
        ),
    cells=dict(
        values=df_table.T.values,
        fill_color= "white",
        line_color= px.colors.sequential.Greys[2],
        font= dict(color= "black", size=13),
        align= "center",
        height= 30
        )
    ),
    col= 1,
    row= 1
)

fig.add_trace(
    go.Choroplethmapbox(
        geojson= geojsn,
        featureidkey= "properties.NAME",
        locations= df_map.county1.values,
        z= df_map.score,
        showlegend=False,
        showscale=True,
        marker_opacity=0.5,
        marker_line_width=0
    ),
    col= 2,
    row= 1
)

fig.update_layout(
    updatemenus= [go.layout.Updatemenu(
            dict(active = 0,
            buttons= list([
                dict(
                    label= county,
                    method= "update",
                    args= [
                        dict(
                            headers= dict(
                                values=list(df_table.columns),
                                fill_color= px.colors.qualitative.G10[4],
                                line_color= px.colors.sequential.Greys[2],
                                font= dict(color= "white", size=15),
                                height= 35
                                ),
                            cells= dict(
                                values= df_table.loc[df_table.county == county].T.values,
                                font= dict(color= "black", size=13),
                                fill_color= "white",
                                line_color= px.colors.sequential.Greys[2],
                                align= "center",
                                height= 30
                                )
                            )
                    ],
                    args2= [
                        dict(
                            z = [df_map.loc[df_map.county1 == county].score.values],
                            locations= [df_map.loc[df_map.county1 == county].county2.values]
                        )
                    ]
                )
                for county in df_table.county.unique().tolist()
            ]),
            direction= "down",
            xanchor= "left",
            x= -0.11,
            yanchor= "top",
            y= 1.1,
            showactive= True),
            dict(
                active = 0,
            buttons= list([
                dict(
                    label= county,
                    method= "update",
                    args= [
                        dict(
                            headers= dict(
                                values=list(df_table.columns),
                                fill_color= px.colors.qualitative.G10[4],
                                line_color= px.colors.sequential.Greys[2],
                                font= dict(color= "white", size=15),
                                height= 35
                                ),
                            cells= dict(
                                values= df_table.loc[df_table.county == county].T.values,
                                font= dict(color= "black", size=13),
                                fill_color= "white",
                                line_color= px.colors.sequential.Greys[2],
                                align= "center",
                                height= 30
                                )
                            )
                    ],
                    args2= [
                        dict(
                            z = [df_map.loc[df_map.county1 == county].score.values],
                            locations= [df_map.loc[df_map.county1 == county].county2.values]
                        )
                    ]
                )
                for county in df_table.county.unique().tolist()
            ]),
            direction= "down",
            xanchor= "left",
            x= -0.11,
            yanchor= "top",
            y= 1.1,
            showactive= True,


            )
        )
    ]
)

fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=4.5, mapbox_center = {"lat": 37, "lon": -120}, height= 600)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

ValueError: 
    Invalid value of type 'builtins.dict' received for the 'active' property of layout.updatemenu
        Received value: {'active': 0, 'buttons': [{'label': 'Alameda', 'method': 'update', 'args': [{'headers': {'values': ['county', 'product_name', 'anzahl', 'rank'], 'fill_color': '#990099', 'line_color': 'rgb(217,217,217)', 'font': {'color': 'white', 'size': 15}, 'height': 35}, 'cells': {'values': array([['Alameda', 'Alameda', 'Alameda', 'Alameda', 'Alameda', 'Alameda',
        'Alameda', 'Alameda', 'Alameda', 'Alameda'],
       ['Banana', 'Bag of Organic Bananas', 'Clementines',
        'Organic Strawberries', 'Granny Smith Apples',
        'Sparkling Mineral Water', 'Apples', '0% Greek Strained Yogurt',
        'Soda', 'Organic Granny Smith Apple'],
       [352, 269, 177, 168, 139, 130, 122, 120, 106, 102],
       [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]], dtype=object), 'font': {'color': 'black', 'size': 13}, 'fill_color': 'white', 'line_color': 'rgb(217,217,

In [153]:
len(df_map.county2.unique())

56

In [163]:
df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= True).county1.head(10).to_list()

['Madera',
 'Amador',
 'Calaveras',
 'Colusa',
 'Contra Costa',
 'Del Norte',
 'El Dorado',
 'Fresno',
 'Glenn',
 'Humboldt']

In [155]:
len(df.county.unique())

57

In [181]:
np.append(np.append([True],df_map.county2.unique() == "Orange"),[True])

array([ True, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True])

In [180]:
len(np.append(np.append([False],df_map.county2.unique() == "Orange"),[False]))

58

In [24]:
df_map.loc[df_map.county2 == "Alpine"].sort_values(by= "score", ascending= False).county1.head(10)

1236    Alameda
Name: county1, dtype: object

In [29]:
pd.concat([df_map.loc[df_map.county2 == "Alpine"],df_map.loc[df_map.county1 == "Alpine"]]).sort_values(by= "score", ascending= False).score.head(10)

45     15
151    13
152    13
153    13
260    12
255    12
256    12
257    12
258    12
259    12
Name: score, dtype: int64

In [41]:
df_map.loc[(df_map.county2 == "Alpine") | (df_map.county1 == "Alpine")].sort_values(by= "score", ascending= False)

,score,county1,county2
45,15,Alpine,San Benito
152,13,Alpine,Mono
153,13,Alpine,Fresno
151,13,Alpine,Sutter
258,12,Alpine,Merced
260,12,Alpine,Glenn
259,12,Alpine,Mariposa
261,12,Alpine,Calaveras
257,12,Alpine,Orange
256,12,Alpine,Santa Cruz


In [35]:
pd.concat([df_map.loc[df_map.county2 == "Alpine"],df_map.loc[df_map.county1 == "Alpine"]]).sort_values(by= "score", ascending= False)[["county1","county2"]].values.flatten() == "Alpine"

array([ True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False, False,  True,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False])

In [39]:
np.delete(pd.concat([df_map.loc[df_map.county2 == "Alpine"],df_map.loc[df_map.county1 == "Alpine"]]).sort_values(by= "score", ascending= False)[["county1","county2"]].values.flatten(),pd.concat([df_map.loc[df_map.county2 == "Alpine"],df_map.loc[df_map.county1 == "Alpine"]]).sort_values(by= "score", ascending= False)[["county1","county2"]].values.flatten() == "Alpine")[0:10]

array(['San Benito', 'Sutter', 'Mono', 'Fresno', 'Glenn', 'Solano',
       'Santa Cruz', 'Orange', 'Merced', 'Mariposa'], dtype=object)

In [26]:
df_map.loc[df_map.county1 == "Alpine"]

,score,county1,county2
45,15,Alpine,San Benito
151,13,Alpine,Sutter
152,13,Alpine,Mono
153,13,Alpine,Fresno
255,12,Alpine,Solano
256,12,Alpine,Santa Cruz
257,12,Alpine,Orange
258,12,Alpine,Merced
259,12,Alpine,Mariposa
260,12,Alpine,Glenn


In [48]:
fig = make_subplots(cols= 2, rows=2, specs= [[dict(type= "table"), dict(type= "choroplethmapbox", rowspan = 2)], [dict(type= "bar"), None]])

fig.add_trace(
  go.Table(
    columnwidth= [2, 4, 2, 2, 2],
    header=dict(
        values=list(df_table.columns)
        ),
    cells=dict(
        values=df_table.T.values,
        height= 28
        )
    ),
    col= 1,
    row= 1
)

# for county in df_map.county2.unique():

buttons1 = []
i = 0 
for county  in df.county.unique():   
    fig.add_trace(
            go.Bar(
                # name=county,                
                y = df_map.loc[(df_map.county2 == county) | (df_map.county1 == county)].sort_values(by= "score", ascending= False).score.head(10),
                x = np.delete(df_map.loc[(df_map.county2 == county) | (df_map.county1 == county)].sort_values(by= "score", ascending= False)[["county1","county2"]].values.flatten(),df_map.loc[(df_map.county2 == county) | (df_map.county1 == county)].sort_values(by= "score", ascending= False)[["county1","county2"]].values.flatten() == county)[0:10],               
                #visible = False,
                showlegend= False
            ),
        col= 1,
        row= 2
        )


fig.add_trace(
    go.Choroplethmapbox(
        geojson= geojsn,
        featureidkey= "properties.NAME",
        locations= df_map.county1.values,
        z= df_map.score,
        showlegend=False,
        showscale=True,
        marker_opacity=0.5,
        marker_line_width=0
    ),
    col= 2,
    row= 1
)

fig.update_layout(
    updatemenus= [
            dict(buttons= list([
                dict(
                    label= county,
                    method= "update",
                    args= [
                        dict(
                            
                            visible = (np.append(np.append([True],df.county.unique() == county),[True])),
                            z = [np.append(df_map.loc[df_map.county2 == county].score.values,df_map.loc[df_map.county1 == county].score.values)],
                            locations= [np.append(df_map.loc[df_map.county2 == county].county1.values,df_map.loc[df_map.county1 == county].county2.values)],
                            headers= dict(
                                values=list(df_table.columns)
                                ),
                            cells= dict(
                                values= df_table.loc[df_table.county == county].T.values,
                                height= 28
                                ),
                            #visible = True
                            )
                    ]
                )
                for county in df_table.county.unique().tolist()
            ]),
            direction= "down",
            xanchor= "left",
            x= -0.11,
            yanchor= "top",
            y= 1.1,
            showactive= True
            ),

            # dict(buttons= list([
            #     dict(
            #         label= county,
            #         method= "update",
            #         args= [
            #             dict(
            #                 visible = (np.append(np.append([True],df.county.unique() == county),[True]))
            #                 # x= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= True).county1.head(10).to_list(),
            #                 # y= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= True).score.head(10).to_list()
            #                 )
            #         ]
            #     )
            #     for county in df.county.unique().tolist()
            # ]),
            # direction= "down",
            # xanchor= "left",
            # x= -0.11,
            # yanchor= "top",
            # y= 0.5,
            # showactive= True
            # ),


            dict(buttons= list([
                dict(
                    label= county,
                    method= "update",
                    args= [
                        dict(
                            visible = (np.append(np.append([True],df.county.unique() == county),[True])),
                            z = [np.append(df_map.loc[df_map.county2 == county].score.values,df_map.loc[df_map.county1 == county].score.values)],
                            locations= [np.append(df_map.loc[df_map.county2 == county].county1.values,df_map.loc[df_map.county1 == county].county2.values)],
                            #visible = True
                            # x= (df_map.county2.unique() == county),
                            # y= (df_map.county2.unique() == county)
                            
                        )
                    ]
                )
                for county in df.county.unique().tolist()
            ]),
            direction= "down",
            xanchor= "left",
            x= 0.45,
            yanchor= "top",
            y= 1.1,
            showactive= True
            )
    ]
)



fig.update_layout(mapbox_style="carto-positron", mapbox_zoom= 5.1, mapbox_center = {"lat": 37, "lon": -120}, height= 850)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df_map.loc[df_map.county2 == "Mono"].sort_values(by= "score", ascending= False).county1.head(10),
y = df_map.loc[df_map.county2 == "Mono"].sort_values(by= "score", ascending= False).score.head(10), 

In [190]:
df_map.loc[df_map.county2 == "Mono"].sort_values(by= "score", ascending= False).county1.head(10).to_list()

['Mariposa',
 'Fresno',
 'Calaveras',
 'Alpine',
 'Glenn',
 'Modoc',
 'Kern',
 'Inyo',
 'Imperial',
 'Madera']

In [171]:
df_map.loc[df_map.county2 == "Orange"]

,score,county1,county2
1,18,Calaveras,Orange
6,17,Fresno,Orange
11,16,Glenn,Orange
23,15,Monterey,Orange
25,15,Mariposa,Orange
30,15,Kern,Orange
58,14,Mono,Orange
203,12,Madera,Orange
210,12,Lake,Orange
217,12,Inyo,Orange


In [107]:
for county in df_map.county2.unique():
    print(county)


Santa Clara
Orange
Glenn
Monterey
Sutter
Siskiyou
Santa Cruz
Mono
Plumas
Kern
Fresno
Ventura
Stanislaus
San Francisco
Mendocino
San Benito
Mariposa
Santa Barbara
San Luis Obispo
Sacramento
Madera
Lake
Los Angeles
Sierra
Sonoma
Solano
Imperial
Tulare
Yolo
Riverside
Tuolumne
Modoc
San Mateo
Nevada
Merced
Marin
Colusa
Calaveras
Yuba
Shasta
Inyo
Contra Costa
San Joaquin
Kings
Del Norte
Napa
San Diego
El Dorado
Amador
Trinity
Tehama
Alpine
Humboldt
Placer
Lassen
San Bernardino


In [117]:
fig = go.Figure()

buttons = []
i = 0

#iterate through dataframes in dict
for county  in df_map.county2.unique():
    #iterate through columns in dataframe (not including the year column)
    # for column in df.drop(columns=['Fuels']):
        #add a bar trace to the figure for the country we are on
    fig.add_trace(go.Bar(
                          name=county,
                          #x axis is "fuels" where dates are stored as per example
                          x = df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= False).county1.head(10),
                          #y axis is the data for the column we are on
                          y = df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= False).score.head(10),
                          #setting only the first country to be visible as default
                          visible = (i==0)
                        )
                     )
        
    #args is a list of booleans that tells the buttons which trace to show on click
    args = [False] * len(df_map.county2.unique())
    args[i] = True
        
        #create a button object for the country we are on
    button = dict(label = county,
                    method = "update",
                    args=[{"visible": args}])
        
        #add the button to our list of buttons
    buttons.append(button)
        
        #i is an iterable used to tell our "args" list which value to set to True
    i+=1
        
fig.update_layout(
    updatemenus=[
        dict(
        #change this to "buttons" for individual buttons
        type="dropdown",
        #this can be "left" or "right" as you like
        direction="down",
        #(1,1) refers to the top right corner of the plot
        x = 1,
        y = 1,
        #the list of buttons we created earlier
        buttons = buttons)
    ],
    #stacked bar chart specified here
    #barmode = "stack",
    #so the x axis increments once per year
    xaxis = dict(dtick = 1))

fig.show()

In [111]:
fig2 = go.Figure(go.Bar(
            x= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= False).county1.head(10),
            y= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= False).score.head(10),
            showlegend= False
        ))
fig2.show()

In [97]:
for county in df_map.county2.unique().tolist():
    print(county)

Santa Clara
Orange
Glenn
Monterey
Sutter
Siskiyou
Santa Cruz
Mono
Plumas
Kern
Fresno
Ventura
Stanislaus
San Francisco
Mendocino
San Benito
Mariposa
Santa Barbara
San Luis Obispo
Sacramento
Madera
Lake
Los Angeles
Sierra
Sonoma
Solano
Imperial
Tulare
Yolo
Riverside
Tuolumne
Modoc
San Mateo
Nevada
Merced
Marin
Colusa
Calaveras
Yuba
Shasta
Inyo
Contra Costa
San Joaquin
Kings
Del Norte
Napa
San Diego
El Dorado
Amador
Trinity
Tehama
Alpine
Humboldt
Placer
Lassen
San Bernardino


In [140]:
df_map.county2.unique() == "Glenn"

array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [185]:
fig = go.Figure()

    
buttons1 = []
i = 0

#iterate through dataframes in dict
for county  in df_map.county2.unique():
    fig.add_trace(go.Bar(
                          name=county,
                          #x axis is "fuels" where dates are stored as per example
                          x = df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= False).county1.head(10),
                          #y axis is the data for the column we are on
                          y = df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= False).score.head(10),
                          #setting only the first country to be visible as default
                          #visible = (i==0)
                          visible= False
                        )
                     )
        
    #args is a list of booleans that tells the buttons which trace to show on click
    # args = [False] * len(df_map.county2.unique())
    # args[i] = True
        
        #create a button object for the country we are on
    button1 = dict(label = county,
                    method = "update",
                    # args=[{"visible": args}]
                    args =[dict(
                            # x= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= True).county1.head(10),
                            # y= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= True).score.head(10),
                            #name = county,
                            visible = (df_map.county2.unique() == county)
                            )]
                    )
        
        #add the button to our list of buttons
    buttons1.append(button1)
        
        #i is an iterable used to tell our "args" list which value to set to True
    i+=1
        
fig.update_layout(
    updatemenus= [
        dict(buttons= buttons1,
            direction= "down",
            xanchor= "left",
            x= -0.11,
            yanchor= "top",
            y= 0.5,
            showactive= True
            ),

    ])
fig.update_layout(mapbox_style="carto-positron", mapbox_zoom= 5.1, mapbox_center = {"lat": 37, "lon": -120}, height= 850)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [122]:
df_map[df_map.county1 == "Tulare"]

,score,county1,county2
262,11,Tulare,Ventura
599,9,Tulare,Tuolumne
980,7,Tulare,Yuba
981,7,Tulare,Yolo


In [124]:
df_map[df_map.county2 == "San Bernardino"]

,score,county1,county2
1529,1,Lassen,San Bernardino
1561,0,San Benito,San Bernardino
1562,0,Sacramento,San Bernardino
1563,0,Riverside,San Bernardino
1564,0,Plumas,San Bernardino
1566,0,Placer,San Bernardino
1567,0,Orange,San Bernardino
1568,0,Nevada,San Bernardino
1569,0,Napa,San Bernardino
1570,0,Monterey,San Bernardino


In [ ]:
dict(buttons= list([
                dict(
                    label= county,
                    method= "update",
                    args= [
                        dict(
                            x= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= True).county1.head(10),
                            y= df_map.loc[df_map.county2 == county].sort_values(by= "score", ascending= True).score.head(10)
                            )
                    ]
                )
                for county in df_map.county2.unique().tolist()
            ]),
            direction= "down",
            xanchor= "left",
            x= -0.11,
            yanchor= "top",
            y= 0.5,
            showactive= True
            ),

In [32]:
df_12= df.groupby("county").agg(order=("order_id","nunique")).reset_index()

In [93]:
df_12.order.median()

7548.0

In [81]:
np.array([2,2,2,2]).reshape(2,2)

array([[2, 2],
       [2, 2]])

In [33]:
liste1 = df_12.loc[df_12.order > df_12.order.median()].county.to_list()

In [34]:
liste2 = df_12.loc[df_12.order <= df_12.order.median()].county.to_list()

In [107]:
liste1.__len__()

28

In [35]:
liste1_score = 0
liste2_score = 0
len1 = 0
len2 = 0
for i in ableitung:
    if (i[1] in liste1 and i[2] in liste1):
        liste1_score = liste1_score + i[0]
        len1 = len1 + 1
    elif (i[1] in liste2 and i[2] in liste2):
        liste2_score = liste2_score + i[0]
        len2 = len2 + 1
liste1_score = liste1_score/len1
liste2_score = liste2_score/len2

In [36]:
liste1_score

10.798941798941799

In [37]:
liste2_score

5.527093596059113

Map nach county paaren oder in gruppen der anliegenden countys

als bar

github in bewerbung

ähnliche counties als map 

2 bis 3 warenkörber der ähnlichsten counties in präsentation einblenden